In [48]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

In [49]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [50]:
torch.cuda.is_available()
device = torch.device("cuda")

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
datapath = '/content/drive/MyDrive/Universitat/ML4NLP/v2.1/'

In [53]:
# There is no train split 
train_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-train-de.tsv'
val_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-dev-de.tsv'
test_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-test_allmasked-de.tsv'

model = 'dbmdz/bert-base-german-europeana-cased'
shuffle = False
tiny = True

In [54]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
tsv_val = load_dataset(val_path)
tsv_test = load_dataset(test_path)
        

In [55]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [56]:
tsv_train = simple_preprocess(tsv_train)
tsv_val = simple_preprocess(tsv_val)
tsv_test = simple_preprocess(tsv_test)

In [57]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

In [58]:
def create_huggingface_file(dataframe, tiny=False):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
            if tiny and idx > 100:
                break
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [59]:
train = create_huggingface_file(tsv_train, tiny=True)
val = create_huggingface_file(tsv_val, tiny=True)
test = create_huggingface_file(tsv_test, tiny=True)

In [60]:
print(len(train))
print(len(val))
print(len(test))

102
102
102


In [61]:
tokenizer = BertTokenizer.from_pretrained(model)

loading file https://huggingface.co/dbmdz/bert-base-german-europeana-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/95d759093a3a1ead95d40f8b0ed2a6dd3fa6c6dbc4ee0c2c392ff72b8b5ea469.34abe0644665e8781c2147427988fa97cf6c1c60ad06f7b356b230f1c5f5d7fe
loading file https://huggingface.co/dbmdz/bert-base-german-europeana-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/dbmdz/bert-base-german-europeana-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/dbmdz/bert-base-german-europeana-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/7cb1e8255a410b68de7e5b11afcec954e38ce8975b556989172adfd6b515c15d.c70618325b9fc2d2d041e439766d360b48a086a8841cc2896322f6b8aefc0225
loading configuration file https://huggingface.co/dbmdz/bert-base-german-europeana-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a516f0

In [62]:
train = train.filter(lambda example, idx: idx != len(train)-1, with_indices=True)
val = val.filter(lambda example, idx: idx != len(val)-1, with_indices=True)
test = test.filter(lambda example, idx: idx != len(test)-1, with_indices=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [63]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [64]:
training_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/Universitat/ML4NLP/bert_europeana_en"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [65]:
if shuffle:
  train_tokenized.shuffle()

In [66]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
# trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer = Trainer(model=model, train_dataset=train_tokenized)
trainer.train()

loading configuration file https://huggingface.co/dbmdz/bert-base-german-europeana-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a516f0d01a57a5da3e377903c8ab946bb07e202f0aadcb224f115addea32a080.d34b61340faaa3122b620c12ba54ab8b1016bfeb2ff97c1d5e80b86808783b7a
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=39, training_loss=0.2448265858185597, metrics={'train_runtime': 345.3742, 'train_samples_per_second': 0.877, 'train_steps_per_second': 0.113, 'total_flos': 15464858450400.0, 'train_loss': 0.2448265858185597, 'epoch': 3.0})

In [67]:
predictions = trainer.predict(test_tokenized)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner, id, words, subwords. If ner, id, words, subwords are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 101
  Batch size = 8


In [33]:
outfilepath = f"/content/drive/MyDrive/e_ML4NLP2/niclas_models_and_data/softlabeltest.tsv"
outfilepath

'/content/drive/MyDrive/e_ML4NLP2/Outputs/softlabeltest.tsv'

In [38]:
print(labels.names)

['B-loc', 'O', 'B-pers', 'I-pers', 'B-org', 'I-org', 'I-loc', 'B-time', 'I-time', 'B-prod', 'I-prod', '_']


In [44]:
outfile = open(outfilepath, "w")
outfile.write("TOKEN\t" + '\t'.join(labels.names))

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_logits = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_logits.append(word_score)
        except IndexError:
            word_score = np.zeros_like(pred[0])
            word_score[1] = 1
            pred_word_logits.append(word_score)

    for token, logits in zip(tinput['words'], pred_word_logits):
        unpacked_logits = "\t".join([str(l) for l in logits])
        outfile.write("\n" + f"{token}\t" + unpacked_logits)

outfile.close()

In [ ]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [ ]:
Trainer(
    model=model,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subwords, words, ner, id. If subwords, words, ner, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1202
  Batch size = 8


(1202, 100, 12)
(1202, 100)


{'eval_accuracy': 0.9908319467554076,
 'eval_f1_macro': 0.759910415468632,
 'eval_f1_micro': 0.9908319467554076,
 'eval_loss': 0.03373601287603378,
 'eval_runtime': 24.7053,
 'eval_samples_per_second': 48.654,
 'eval_steps_per_second': 6.112}